# Chap06 - 텍스트와 시퀀스를 위한 딥러닝

## Goals

- 텍스트 데이터를 유용한 형태로 전처리하는 방법
- 순환 신경망(RNN)을 사용하는 방법
- 1D 컨브넷을 사용한 시퀀스 데이터의 처리

## Intro

- 시퀀스(sequence) 데이터를 처리하는 기본적인 딥러닝 모델을 **순환 신경망(RNN, Recurrent Neural Network)**과 **1D 컨브넷(1D convnet)** 두 가지다.
  - 1D 컨브넷은 2D 컨브넷의 1차원 버전이다.
- 다음과 같은 예제에 사용할 수 있다.
  - 문서 분류나 시계열 분류. → 글의 주제나 책의 저자 식별하기
  - 시계열 비교. → 두 문서나 두 주식 가격이 얼마나 밀접하게 관련이 있는지 추정하기
  - 시퀀스-투-시퀀스 학습. → 영어 문장을 한국어로 번역
  - 감성 분석 → 영화 리뷰 긍정/부정 분류하기
  - 시계열 예측 → 향후 날씨 예측하기

## 6.1 텍스트 데이터 다루기

- 텍스트는 가장 흔한 시퀀스 형태의 데이터다.
  - 텍스트는 단어의 시퀀스나 문자의 시퀀스로 이해할 수 있다.
  - 보통 단어 수준으로 작업하는 경우가 많다.

- 시퀀스 처리용 딥러닝 모델은 텍스트를 사용하여 기초적인 자연어 이해(NLU, natural language understanding) 문제를 처리할 수 있다.
- 이러한 모델은 문자 언어(written language)에 대한 **통계적 구조**를 만들어 간단한 텍스트 문제를 해결한다.
- 자연어 처리(NLP, natural language processing)를 위한 딥러닝은 **단어, 문장, 문단**에 적용한 패턴인식이다.
- 딥러닝 모델의 입력으로 텍스트 원본을 사용하지 못하기 때문에 **텍스트 → 텐서** 로 변환해줘야 한다 → **텍스트 벡터화(vectorizing text)**
- 텍스트 벡터화에는 여러가지 방식이 있다.
  - 텍스트를 **단어(word)**로 나누고 각 단어를 하나의 벡터로 변환한다.
  - 텍스트를 **문자(character)** 로 나누고 각 문자를 하나의 벡터로 변환한다.
  - 텍스트에서 단어나 문자의 **n-gram**을 추출하여 각 n-gram을 하나의 벡터로 변환한다.
- 텍스트를 나누는 단위(단어, 문자, n-gram)을 **토큰(token)**이라고 하며, 토큰으로 나누는 작업을 **토큰화(tokenization)** 라고 한다.
- 모든 텍스트 벡터화 과정은 어떤 종류의 토큰화를 적용하고 **생성된 토큰에 벡터를 연결하는 것** 으로 이루어진다.
  - 이 벡터는 시퀀스 텐서로 묶여져서 신경망에 주입된다.
- 토큰과 벡터를 연결하는 방법은 여러가지가 있는데, 대표적인 두 가지는 다음과 같다.
  - **원-핫 인코딩(one-hot encoding)**
  - **토큰 임베딩(token embedding)** = **단어 임베딩(word embedding)**

### 6.1.1 단어와 문자의 원-핫 인코딩

원-핫 인코딩은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법입니다. [3장에서 IMDB](https://github.com/ExcelsiorCJH/Deep-Learning-with-Python/blob/master/Chap03-Getting_started_with_neural_networks/3.4-classifying_movie_reviews.ipynb)와 [로이터 예제](https://github.com/ExcelsiorCJH/Deep-Learning-with-Python/blob/master/Chap03-Getting_started_with_neural_networks/3.5-classifying_newswires.ipynb)에서 이를 보았습니다(단어의 원-핫 인코딩을 사용했습니다). 

모든 단어에 **고유한 정수 인덱스**를 부여하고 이 정수 인덱스 i를 크기가 N(어휘 사전의 크기)인 이진 벡터로 변환합니다. 이 벡터는 **i번째 원소만 1이고 나머지는 모두 0**입니다.

물론 원-핫 인코딩은 문자 수준에서도 적용할 수 있습니다. 원-핫 인코딩이 무엇이고 어떻게 구현하는지 명확하게 설명하기 위해 단어와 문자에 대한 간단한 예를 만들었습니다.

#### 단어 수준의 원-핫 인코딩(간단한 예):

In [2]:
import numpy as np

# 초기 데이터: 각 원소가 샘플입니다
# (이 예에서 하나의 샘플이 하나의 문장입니다. 하지만 문서 전체가 될 수도 있습니다)
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 데이터에 있는 모든 토큰의 인덱스를 구축한다.
token_index = {}
for sample in samples:
    # split() 메서드를 사용해 샘플을 토큰으로 나눈다.
    # 실전에서는 구둣점과 특수 문자도 사용합니다.
    for word in sample.split():
        if word not in token_index:
            # 단어마다 고유한 인덱스를 할당한다.
            token_index[word] = len(token_index) + 1
            # 인덱스 0은 사용하지 않는다.
            
# 샘플을 벡터로 변환
# 각 샘플에서 max_length 까지 단어만 사용
max_length = 10

# 결가를 저장할 배열
results = np.zeros((len(samples), max_length, max(token_index.values())+1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [7]:
token_index

{'The': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'the': 5,
 'mat.': 6,
 'dog': 7,
 'ate': 8,
 'my': 9,
 'homework.': 10}

In [9]:
print('results.shape :', results.shape)
results

results.shape : (2, 10, 11)


array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

#### 문자 수준 원-핫 인코딩(간단한 예)

In [16]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # 출력 가능한 모든 아스키(ASCII) 문자
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values())+1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

In [19]:
print(token_index)

{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36, 'A': 37, 'B': 38, 'C': 39, 'D': 40, 'E': 41, 'F': 42, 'G': 43, 'H': 44, 'I': 45, 'J': 46, 'K': 47, 'L': 48, 'M': 49, 'N': 50, 'O': 51, 'P': 52, 'Q': 53, 'R': 54, 'S': 55, 'T': 56, 'U': 57, 'V': 58, 'W': 59, 'X': 60, 'Y': 61, 'Z': 62, '!': 63, '"': 64, '#': 65, '$': 66, '%': 67, '&': 68, "'": 69, '(': 70, ')': 71, '*': 72, '+': 73, ',': 74, '-': 75, '.': 76, '/': 77, ':': 78, ';': 79, '<': 80, '=': 81, '>': 82, '?': 83, '@': 84, '[': 85, '\\': 86, ']': 87, '^': 88, '_': 89, '`': 90, '{': 91, '|': 92, '}': 93, '~': 94, ' ': 95, '\t': 96, '\n': 97, '\r': 98, '\x0b': 99, '\x0c': 100}


In [17]:
print('results.shape :', results.shape)
results

results.shape : (2, 50, 101)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

**케라스**에는 원본 텍스트 데이터를 단어 또는 문자 수준의 **원-핫 인코딩으로 변환해주는 유틸리티**가 있습니다. 특수 문자를 제거하거나 빈도가 높은 N개의 단어만을 선택(입력 벡터 공간이 너무 커지지 않도록 하기 위한 일반적인 제한 방법입니다)하는 등 여러 가지 중요한 기능들이 있기 때문에 이 유틸리티를 사용하는 것이 좋습니다.

#### 케라스를 사용한 단어 수준의 원-핫 인코딩: [[링크]](https://keras.io/preprocessing/text/) 참고

In [20]:
import keras 

keras.__version__

'2.2.4'

In [32]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 가장 빈도가 높은 1,000개의 단어만 선택하도록 Tokenizer 객체를 만든다.
tokenizer = Tokenizer(num_words=1000)
# 단어 인덱스를 구축한다.
tokenizer.fit_on_texts(samples)

# 문자열을 정수 인덱스의 리스트로 변환
# sequences = [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
sequences = tokenizer.texts_to_sequences(samples)

# 직접 원-핫 이진 벡터 표현을 얻을 수 있다.
# 원-핫 인코딩 외에 다른 벡터화 방법들도 제공한다.
# mode : one of "binary", "count", "tfidf", "freq".
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 계산된 단어 인덱스를 구한다.
word_index = tokenizer.word_index
print('Found {:d} unique tokens.'.format(len(word_index)))
print('word_index :\n', word_index)
one_hot_results

Found 9 unique tokens.
word_index :
 {'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

원-핫 인코딩의 변종 중 하나는 **원-핫 해싱(one-hot hashing)** 기법입니다. 
- 이 방식은 어휘 사전에 있는 고유한 토큰의 수가 너무 커서 모두 다루기 어려울 때 사용합니다. 
- 각 단어에 명시적으로 인덱스를 할당하고 이 인덱스를 딕셔너리에 저장하는 대신에 **단어를 해싱하여 고정된 크기의 벡터로 변환**합니다. 
- 일반적으로 간단한 **해싱 함수**를 사용합니다. 
- 이 방식의 주요 장점은 **명시적인 단어 인덱스가 필요 없기 때문에 메모리를 절약하고 온라인 방식으로 데이터를 인코딩**할 수 있습니다(전체 데이터를 확인하지 않고 토큰을 생성할 수 있습니다). 
- 한 가지 단점은 **해시 충돌(hash collision)**입니다. 
    - 두 개의 단어가 같은 해시를 만들면 이를 바라보는 머신 러닝 모델은 단어 사이의 차이를 인식하지 못합니다. 
    - 해싱 공간의 차원이 해싱될 고유 토큰의 전체 개수보다 훨씬 크면 해시 충돌의 가능성은 감소합니다.

#### 해싱 기법을 사용한 단어 수준의 원-핫 인코딩(간단한 예):

In [35]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 단어를 크기가 1,000인 벡터로 저장
# 1,000개(또는 그이상)의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될 것이다.
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 단어를 해싱하여 0과 1,000 사이의 랜덤한 정수 인덱스로 변환한다.
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

In [37]:
print('results.shape :', results.shape)
results

results.shape : (2, 10, 1000)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

#### 케라스를 이용한 해싱 기법을 사용한 단어 수준의 원-핫 인코딩:

In [18]:
from keras.preprocessing.text import hashing_trick

dimensionality = 1000
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

hashing_results = [hashing_trick(sample, dimensionality) for sample in samples]
hashing_results

[[615, 555, 806, 866, 615, 796], [615, 640, 168, 664, 402]]

In [21]:
# sequence to matrix
results = np.zeros((len(hashing_results), dimensionality))

for i, hashing_result in enumerate(hashing_results):
    results[i][hashing_result] = 1.

In [25]:
print('results.shape :', results.shape)
results

results.shape : (2, 1000)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

---

### 6.1.2 단어 임베딩 사용하기

단어와 벡터를 연관짓는 강력하고 인기 있는 또 다른 방법은 **단어 임베딩**이라는 밀집 **단어 벡터**를 사용하는 것입니다. 
- 원-핫 인코딩으로 만든 벡터는 희소하고(대부분 0으로 채워짐) 고차원입니다(어휘 사전에 있는 단어의 수와 차원이 같습니다). 
- 반면 **단어 임베딩**은 **저차원의 실수형 벡터**입니다(희소 벡터의 반대인 밀집 벡터입니다). 

원-핫 인코딩으로 얻은 단어 벡터와 달리 **단어 임베딩은 데이터로부터 학습**됩니다. 
- 보통 256차원, 512차원 또는 큰 어휘 사전을 다룰 때는 1,024차원의 단어 임베딩을 사용합니다. 
- 반면 원-핫 인코딩은 (20,000개의 토큰으로 이루어진 어휘 사전을 만들려면) 20,000차원 또는 그 이상의 벡터일 경우가 많습니다. 
- 따라서 단어 임베딩이 더 많은 정보를 적은 차원에 저장합니다.

![](./images/embedding.png)

#### `Embedding` 레이어를 사용하여 단어 임베딩 학습하기

단어와 밀집 벡터를 연관짓는 가장 간단한 방법은 **랜덤하게 벡터를 선택**하는 것입니다. 
- 이 방식의 문제점은 임베딩 **공간이 구조적이지 않다는 것**입니다. 
- 예를 들어 'accurate'와 'exact' 단어가 대부분 문장에서 비슷한 의미로 사용되지만 완전히 다른 임베딩을 가지게 됩니다. 
- 심층 신경망이 이런 임의의 구조적이지 않은 임베딩 공간을 이해하기는 어렵습니다.

단어 벡터 사이에 조금 더 추상적이고 기하학적인 관계를 얻으려면 단어 사이에 있는 **의미 관계를 반영**해야 합니다. 

단어 임베딩은 언어를 기하학적 공간에 매핑하는 것입니다. 
- 예를 들어 잘 구축된 임베딩 공간에서는 동의어가 비슷한 단어 벡터로 임베딩될 것입니다. 
- 일반적으로 두 단어 벡터 사이의 거리(L2 거리)는 이 **단어 사이의 의미 거리와 관계**되어 있습니다(멀리 떨어진 위치에 임베딩된 단어의 의미는 서로 다르고 반면 비슷한 단어들은 가까이 임베딩됩니다). 
- 거리외에 임베딩 공간의 특정 방향도 의미를 가질 수 있습니다.

실제 단어 임베딩 공간에서 의미 있는 기하학적 변환의 일반적인 예는 '성별' 벡터와 '복수(plural)' 벡터입니다. 
- 예를 들어 'king' 벡터에 'female' 벡터를 더하면 'queen' 벡터가 됩니다. 
- 'plural' 벡터를 더하면 'kings'가 됩니다. 
- 단어 임베딩 공간은 전형적으로 이런 해석 가능하고 잠재적으로 유용한 수천 개의 벡터를 특성으로 가집니다.


<img src="./images/w2v.png" height="60%" width="60%"/>


사람의 언어를 완벽하게 매핑해서 어떤 자연어 처리 작업에도 사용할 수 있는 이상적인 단어 임베딩 공간이 있을까요? 아마도 가능하겠지만 아직까지 이런 종류의 공간은 만들지 못했습니다. 사람의 언어에도 그런 것은 없습니다. 세상에는 많은 다른 언어가 있고 언어는 특정 문화와 환경을 반영하기 때문에 서로 동일하지 않습니다. 

실제로 좋은 단어 임베딩 공간을 만드는 것은 문제에 따라 크게 달라집니다. 
- 영어로 된 영화 리뷰 감성 분석 모델을 위한 완벽한 단어 임베딩 공간은 영어로 된 법률 문서 분류 모델을 위한 완벽한 임베딩 공간과 다를 것 같습니다. 
- 특정 의미 관계의 중요성이 작업에 따라 다르기 때문입니다.

따라서 **새로운 작업에는 새로운 임베딩**을 학습하는 것이 타당합니다. 

다행히 역전파를 사용해 쉽게 만들 수 있고 **케라스**를 사용하면 더 쉽습니다. [**Embedding 층**](https://keras.io/layers/embeddings/)의 가중치를 학습하면 됩니다.

In [27]:
from keras.layers import Embedding

# Embedding 레이어는 적어도 두 개의 매개변수를 받는다.
# - 가능한 토큰의 개수 (size of vocabulary)
# - 임베딩 차원
embedding_layer = Embedding(1000, 64)

`Embedding` 층을 (특정 단어를 나타내는) 정수 인덱스를 밀집 벡터로 매핑하는 딕셔너리로 이해하는 것이 가장 좋습니다. 정수를 입력으로 받아 내부 딕셔너리에서 이 정수에 연관된 벡터를 찾아 반환합니다. 딕셔너리 탐색은 효율적으로 수행됩니다.
- TensorFlow를 백엔드로 사용할 경우 `tf.nn.embedding_lookup()` 함수를 사용하여 병렬처리된다.



$$
\text{Word Index} \rightarrow \text{Embedding layer} \rightarrow \text{Corresponding word vector}
$$

<img src="./images/embedding02.jpg" height="60%" width="60%"/>

`Embedding` 층은 크기가 `(samples, sequence_length)`인 2D 정수 텐서를 입력으로 받습니다. 각 샘플은 정수의 시퀀스입니다. 가변 길이의 시퀀스를 임베딩할 수 있습니다. 
- 예를 들어 위 예제의 `Embedding` 층에 `(32, 10)` 크기의 배치(길이가 10인 시퀀스 32개로 이루어진 배치)나 `(64, 15)` 크기의 배치(길이가 15인 시퀀스 64개로 이루어진 배치)를 주입할 수 있습니다. 
    - 배치에 있는 모든 시퀀스는 길이가 같아야 하므로(하나의 텐서에 담아야 하기 때문에) 작은 길이의 시퀀스는 0으로 패딩되고 길이가 더 긴 시퀀스는 잘립니다.

`Embedding` 층은 크기가 `(samples, sequence_length, embedding_dimensionality)`인 3D 실수형 텐서를 반환합니다.
- 이런 3D 텐서는 RNN 층이나 1D 합성곱 층에서 처리됩니다(둘 다 이어지는 절에서 소개하겠습니다).

`Embedding` 층의 객체를 생성할 때 가중치(토큰 벡터를 위한 내부 딕셔너리)는 다른 층과 마찬가지로 **랜덤하게 초기화**됩니다. 
- 훈련하면서 이 단어 벡터는 역전파를 통해 점차 조정되어 이어지는 모델이 사용할 수 있도록 임베팅 공간을 구성합니다. 
- 훈련이 끝나면 임베딩 공간은 특정 문제에 특화된 구조를 많이 가지게 됩니다.

이를 익숙한 IMDB 영화 리뷰 감성 예측 문제에 적용해 보죠. 
- 먼저 데이터를 준비합니다. 
- 영화 리뷰에서 가장 빈도가 높은 10,000개의 단어를 추출하고(처음 이 데이터셋으로 작업했던 것과 동일합니다) 리뷰에서 20개 단어 이후는 버립니다. 
- 이 네트워크는 10,000개의 단어에 대해 8 차원의 임베딩을 학습하여 정수 시퀀스 입력(2D 정수 텐서)를 임베딩 시퀀스(3D 실수형 텐서)로 바꿀 것입니다. 
- 그 다음 이 텐서를 2D로 펼쳐서 분류를 위한 Dense 층을 훈련하겠습니다.

In [34]:
from keras.datasets import imdb
from keras import preprocessing

# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이 (가장 빈번한 max_features 개의 단어만 사용합니다)
maxlen = 20

# 정수 리스트로 데이터를 로드한다.
(train_x, train_y), (test_x, test_y) = imdb.load_data(num_words=max_features)

print("train_x's shape (before transform) :", train_x.shape)
print("test_x's shape (before transform) :", test_x.shape)

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환한다
train_x = preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)

print("train_x's shape (after transform) :", train_x.shape)
print("test_x's shape (after transform) :", test_x.shape)

train_x's shape (before transform) : (25000,)
test_x's shape (before transform) : (25000,)
train_x's shape (after transform) : (25000, 20)
test_x's shape (after transform) : (25000, 20)


In [36]:
from keras import layers, models
from keras import backend as K

K.clear_session()

model = models.Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 
# 펼치기 위해 Embedding 층에 input_length를 지정한다.
model.add(layers.Embedding(10000, 8, input_length=maxlen))
# Embedding 층의 출력은 (samples, maxlen, 8)이 된다.

# 3D 임베딩 텐서를 (samples, maxlen * 8) 형태의 2D 텐서로 펼친다.
model.add(layers.Flatten())

# 분류기를 추가한다.
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', 
              metrics=['acc'])

model.summary()

history = model.fit(train_x, train_y, 
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.6743 - acc: 0.6073 - val_loss: 0.6291 - val_acc: 0.6876
Epoch 2/10
20000/20000 [==============================] - 1s 40us/step - loss: 0.5511 - acc: 0.7482 - val_loss: 0.5302 - val_acc: 0.7230
Epoch 3/10
20000/20000 [==============================] - 1s 46us/step - loss: 

약 75% 정도의 검증 정확도가 나옵니다. 리뷰에서 20개의 단어만 사용한 것치고 꽤 좋은 결과입니다. 

하지만 임베딩 시퀀스를 펼치고 하나의 `Dense` 층을 훈련했으므로 입력 시퀀스에 있는 **각 단어를 독립적**으로 다루었습니다. 단어 사이의 **관계나 문장의 구조를 고려하지 않았습니다**(예를 들어 이 모델은 “this movie is a bomb”와 “this movie is the bomb”를 부정적인 리뷰로 동일하게 다룰 것입니다). 

각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 **순환 층**이나 **1D 합성곱 층**을 추가하는 것이 좋습니다. 다음 절에서 이에 관해 집중적으로 다루겠습니다.

#### Embedding 결과 확인해보기

In [38]:
embedding_output = model.layers[0].output

embedding_model = models.Model(inputs=model.input, outputs=embedding_output)
embedding_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1_input (InputLaye (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 8)             80000     
Total params: 80,000
Trainable params: 80,000
Non-trainable params: 0
_________________________________________________________________


In [51]:
word_to_index = imdb.get_word_index()
index_to_word = {idx: word for word, idx in word_to_index.items()}

sample1 = train_x[0]
sample1_to_text = [index_to_word[idx] for idx in sample1]

sample2 = train_x[100]
sample2_to_text = [index_to_word[idx] for idx in sample2]

In [68]:
print('sample1:\n{}'.format(sample1))
print('sample1_to_text:\n{}'.format(sample1_to_text))

sample1:
[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
   15   16 5345   19  178   32]
sample1_to_text:
['their', 'with', 'her', 'nobody', 'most', 'that', 'with', "wasn't", 'to', 'with', 'armed', 'acting', 'watch', 'an', 'for', 'with', 'heartfelt', 'film', 'want', 'an']


In [69]:
embedding_sample1 = embedding_model.predict(sample1.reshape([1,20]))
embedding_sample1.shape

(1, 20, 8)